In [47]:
import numpy as np
import copy
from itertools import combinations
np.set_printoptions(suppress=True, linewidth=np.nan)
from util import build_tableau, real_index, pivoting, basic2vertex, correct_all_tableau, minimum_ratio_test
# https://graphonline.ru/en/

# Adjacent Graph with simplex method
Consiste en visitar todos los vertices. Esto sucede cuando hay elementos positivos en la fila cero y cuando hay empate en el test de minimo ratio. Buscar todas las posibles combinaciones lineales al inicio para visitar todos los vertices. Es posible visitar a todos con solo una sola solución básica factible inicial.

### Utilities

In [48]:
def minimum_ratio_test_multiple(col: np.ndarray, b: np.ndarray) -> int:
    map_test: dict[float, list[int]] = {}
    min_key = np.Inf
    for k in range(len(b)):
        if col[k] > 0:
            test = round(b[k] / col[k], 6)
            if test in map_test:
                map_test[test].append(k)
            else:
                map_test[test] = [k]
            min_key = min(min_key, test)
    return map_test[min_key] if min_key != np.Inf else None


### Adjacent Graph

In [49]:
def adjacent_graph(tableau: np.ndarray, basic_var: list[int], row: int = None, col: int = None) -> None:
    vertex = basic2vertex(tableau, basic_var)    
    if (tableau[0, :-1] > 0).any():
        # recursive simplex for each column candidate
        for col in np.where(tableau[0, :-1] > 0)[0]:
            # minimum ratio test
            multi_row = minimum_ratio_test_multiple(col=tableau[1:, col], b=tableau[1:, -1])
            if multi_row != None:
                for row in multi_row:
                    # temp array
                    temp_tableau = np.copy(tableau)
                    temp_basic_var = copy.copy(basic_var)
                    # pivoting
                    pivoting(temp_tableau, row + 1, col)
                    # swap row with col
                    temp_basic_var[row] = col
                    # recursive proccess
                    print(f"vertex = {vertex}, x_B = {real_index(basic_var)}\n{tableau}")
                    adjacent_graph(temp_tableau, temp_basic_var, row, col)
            else:
                print("The model is unbounded")
    else:
        print(f"vertex = {vertex}\tx_B = {real_index(basic_var)}\n{tableau}")
        print("end proccess")

In [50]:
A = [[1, 1, 1, 0, 0, 0],
     [3, 1, 0, 1, 0, 0],
     [1, 0, 0, 0, 1, 0],
     [0, 1, 0, 0, 0, 1.]]

b = [9, 18, 7, 6.]

c = [3, 2, 0, 0, 0, 0.]

basic_var = [2, 3, 4, 5]

tableau = build_tableau(A, b, c)
adjacent_graph(tableau, basic_var, [])

vertex = [0. 0.], x_B = [3, 4, 5, 6]
[[ 3.  2.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  0.  0.  0.  9.]
 [ 3.  1.  0.  1.  0.  0. 18.]
 [ 1.  0.  0.  0.  1.  0.  7.]
 [ 0.  1.  0.  0.  0.  1.  6.]]
vertex = [6. 0.], x_B = [3, 1, 5, 6]
[[  0.           1.           0.          -1.           0.           0.         -18.        ]
 [  0.           0.66666667   1.          -0.33333333   0.           0.           3.        ]
 [  1.           0.33333333   0.           0.33333333   0.           0.           6.        ]
 [  0.          -0.33333333   0.          -0.33333333   1.           0.           1.        ]
 [  0.           1.           0.           0.           0.           1.           6.        ]]
vertex = [4.5 4.5]	x_B = [2, 1, 5, 6]
[[  0.    0.   -1.5  -0.5   0.    0.  -22.5]
 [  0.    1.    1.5  -0.5   0.    0.    4.5]
 [  1.    0.   -0.5   0.5   0.    0.    4.5]
 [  0.    0.    0.5  -0.5   1.    0.    2.5]
 [  0.    0.   -1.5   0.5   0.    1.    1.5]]
end proccess
vertex = [0. 0.], x_B 

In [51]:
A = [[1, 1, 1, 1, 0],
     [1, 0, 0, 0, 1.]]

b = [1, 1.]

c = [0, 0, 1, 0, 0.]

# basic_var = [3, 4]
basic_var_list = [[3, 4], [1, 4], [0, 4], [0, 1], [0, 3]]

for basic_var in basic_var_list:
     print("------- New simplex proccess -------")
     tableau = build_tableau(A, b, c)
     correct_all_tableau(tableau, basic_var)
     adjacent_graph(tableau, basic_var)

------- New simplex proccess -------
All tableau corrected
vertex = [0. 0. 0.], x_B = [4, 5]
[[0. 0. 1. 0. 0. 0.]
 [1. 1. 1. 1. 0. 1.]
 [1. 0. 0. 0. 1. 1.]]
vertex = [0. 0. 1.]	x_B = [3, 5]
[[-1. -1.  0. -1.  0. -1.]
 [ 1.  1.  1.  1.  0.  1.]
 [ 1.  0.  0.  0.  1.  1.]]
end proccess
------- New simplex proccess -------
All tableau corrected
vertex = [0. 1. 0.], x_B = [2, 5]
[[0. 0. 1. 0. 0. 0.]
 [1. 1. 1. 1. 0. 1.]
 [1. 0. 0. 0. 1. 1.]]
vertex = [0. 0. 1.]	x_B = [3, 5]
[[-1. -1.  0. -1.  0. -1.]
 [ 1.  1.  1.  1.  0.  1.]
 [ 1.  0.  0.  0.  1.  1.]]
end proccess
------- New simplex proccess -------
All tableau corrected
vertex = [1. 0. 0.], x_B = [1, 5]
[[ 0.  0.  1.  0.  0.  0.]
 [ 1.  1.  1.  1.  0.  1.]
 [ 0. -1. -1. -1.  1.  0.]]
vertex = [0. 0. 1.]	x_B = [3, 5]
[[-1. -1.  0. -1.  0. -1.]
 [ 1.  1.  1.  1.  0.  1.]
 [ 1.  0.  0.  0.  1.  1.]]
end proccess
------- New simplex proccess -------
All tableau corrected
vertex = [ 1. -0.  0.], x_B = [1, 2]
[[ 0.  0.  1.  0.  0.  0.]
 [ 1

In [52]:
A = [[-2, 6, 1, 0, 0],
     [2, 4., 0, 1, 0],
     [1, 0., 0, 0, 1]]

b = [0, 10, 4.]

c = [-1, 1, 0, 0, 0.]

# un vertice suficiente para visitarlos a todos
basic_var_list = [[0, 3, 2]] # respetar el orden en la tabla

for basic_var in basic_var_list:
     print("------- New simplex proccess -------")
     tableau = build_tableau(A, b, c)
     correct_all_tableau(tableau, basic_var)
     adjacent_graph(tableau, basic_var, [])

------- New simplex proccess -------
All tableau corrected
vertex = [4. 0.], x_B = [1, 4, 3]
[[ 0.  1.  0.  0.  1.  4.]
 [ 1.  0.  0.  0.  1.  4.]
 [ 0.  4.  0.  1. -2.  2.]
 [ 0.  6.  1.  0.  2.  8.]]
vertex = [4.  0.5], x_B = [1, 2, 3]
[[ 0.    0.    0.   -0.25  1.5   3.5 ]
 [ 1.    0.    0.    0.    1.    4.  ]
 [ 0.    1.    0.    0.25 -0.5   0.5 ]
 [ 0.    0.    1.   -1.5   5.    5.  ]]
vertex = [3. 1.], x_B = [1, 2, 5]
[[ 0.   0.  -0.3  0.2  0.   2. ]
 [ 1.   0.  -0.2  0.3  0.   3. ]
 [ 0.   1.   0.1  0.1  0.   1. ]
 [ 0.   0.   0.2 -0.3  1.   1. ]]
vertex = [0. 0.]	x_B = [4, 2, 5]
[[-0.66666667  0.         -0.16666667  0.          0.          0.        ]
 [ 3.33333333  0.         -0.66666667  1.          0.         10.        ]
 [-0.33333333  1.          0.16666667  0.          0.          0.        ]
 [ 1.          0.          0.          0.          1.          4.        ]]
end proccess
vertex = [3. 1.], x_B = [1, 2, 5]
[[ 0.   0.  -0.3  0.2  0.   2. ]
 [ 1.   0.  -0.2  0.3  0

In [53]:
def simplex_cycle(tableau: np.ndarray, basic_var: list[int]) -> np.ndarray:
    path: list = [basic_var[:]]
    print(f"x_B = {real_index(basic_var)}\n{tableau}")
    while (tableau[0, :-1] > 0).any():
        # max criterion
        c_max = np.argmax(tableau[0, :-1])
        # minimum ratio test

        if (r_min := minimum_ratio_test(col=tableau[1:, c_max], b=tableau[1:, -1])) != None:
            # pivoting
            pivoting(tableau, r_min + 1, c_max)
            # swap row with col
            basic_var[r_min] = c_max

            vertex = basic2vertex(tableau, basic_var)
            print(f"vertex = {vertex}\tx_B = {real_index(basic_var)}\n{tableau}\npivoted = {r_min, c_max}")

            if basic_var in path:
                path.append(basic_var[:])
                print(f"There's a cycle {basic_var}")
                break
            path.append(basic_var[:])
        else:
            print("The model is unbounded")
            break

    print(list(map(lambda x: real_index(x), path)))
    return tableau


In [54]:
A = [[1/2, -11/2, -5/2, 9, 1, 0],
     [1/2, -3/2, -1/2, 1., 0, 1]]

b = [0.0, 0.0]

c = [10, -57, -9, -24, 0.0, 0.0]

m = len(b)
n = len(c) - m
tableau = build_tableau(A, b, c)
all_basic_var = map(lambda x: list(x), combinations(range(n + m), m))
for basic_var in all_basic_var:
     tableau = np.copy(tableau)
     correct_all_tableau(tableau, basic_var)
     print("------- New simplex proccess -------")
     simplex_cycle(tableau, basic_var)
     print("End process")



# basic_var = [4, 5]
# tableau = build_tableau(A, b, c)
# simplex(tableau, basic_var)


All tableau corrected
------- New simplex proccess -------
x_B = [1, 2]
[[  0.     0.    14.5  -98.    -6.75 -13.25   0.  ]
 [  1.     0.     0.5   -4.    -0.75   2.75   0.  ]
 [  0.     1.     0.5   -2.    -0.25   0.25   0.  ]]
vertex = [0. 0. 0. 0.]	x_B = [3, 2]
[[-29.    0.    0.   18.   15.  -93.    0. ]
 [  2.    0.    1.   -8.   -1.5   5.5   0. ]
 [ -1.    1.    0.    2.    0.5  -2.5   0. ]]
pivoted = (0, 2)
vertex = [0. 0. 0. 0.]	x_B = [3, 4]
[[-20.    -9.     0.     0.    10.5  -70.5    0.  ]
 [ -2.     4.     1.     0.     0.5   -4.5    0.  ]
 [ -0.5    0.5    0.     1.     0.25  -1.25   0.  ]]
pivoted = (1, 3)
vertex = [0. 0. 0. 0.]	x_B = [5, 4]
[[ 22.  -93.  -21.    0.    0.   24.    0. ]
 [ -4.    8.    2.    0.    1.   -9.    0. ]
 [  0.5  -1.5  -0.5   1.    0.    1.    0. ]]
pivoted = (0, 4)
vertex = [0. 0. 0. 0.]	x_B = [5, 6]
[[ 10.  -57.   -9.  -24.    0.    0.    0. ]
 [  0.5  -5.5  -2.5   9.    1.    0.    0. ]
 [  0.5  -1.5  -0.5   1.    0.    1.    0. ]]
pivoted = (

In [55]:
A = [[1 / 3, 1, -1 / 3, -2, 1, 0],
     [-2.0, -9.0, 1.0, 9.0, 0, 1]]

b = [0.0, 0.0]

c = [2, 3, -1, -12, 0, 0.0]

m = len(b)
n = len(c) - m
tableau = build_tableau(A, b, c)
all_basic_var = map(lambda x: list(x), combinations(range(n + m), m))
for basic_var in all_basic_var:
     tableau = np.copy(tableau)
     correct_all_tableau(tableau, basic_var)
     print("Start process")
     simplex_cycle(tableau, basic_var)
     print("End process")



# basic_var = [4, 5]
# tableau = build_tableau(A, b, c)
# simplex(tableau, basic_var)


All tableau corrected
Start process
x_B = [1, 2]
[[  0.           0.           2.           3.         -12.          -1.           0.        ]
 [  1.           0.          -2.          -9.           9.           1.           0.        ]
 [ -0.           1.           0.33333333   1.          -2.          -0.33333333  -0.        ]]
vertex = [ 0.  0.  0. -0.]	x_B = [1, 4]
[[ 0.         -3.          1.          0.         -6.          0.          0.        ]
 [ 1.          9.          1.          0.         -9.         -2.          0.        ]
 [-0.          1.          0.33333333  1.         -2.         -0.33333333 -0.        ]]
pivoted = (1, 3)
vertex = [ 0.  0.  0. -0.]	x_B = [3, 4]
[[ -1.         -12.           0.           0.           3.           2.           0.        ]
 [  1.           9.           1.           0.          -9.          -2.           0.        ]
 [ -0.33333333  -2.           0.           1.           1.           0.33333333  -0.        ]]
pivoted = (0, 2)
vertex = 

### CM4E1 Curso: Práctica Calificada 3

##### 2. Determinar el grafo de adyacencia del siguiente LO

\begin{align*}
\max \quad & x_1 + x_2 \\
\text { s.t. } \quad  &x_1& + 2 x_2& + x_{3}& {} {}& {} {}&=5&, \\
                      &x_1& - x_2&  {} {}& + x_{4}& {} {}&=2&, \\
                     &-x_1& + 3 x_2&  {} {}& {} {}& + x_{5}&=0&, \\
\end{align*}
$$x_1 \ge 0, x_2 \ge 0, x_3 \ge 0, x_4 \geq 0, x_{3} \geq 0$$

In [56]:
A = [[1, 2., 1, 0, 0],
     [1, -1, 0, 1, 0],
     [-1, 3, 0, 0, 1]]

b = [5, 2, 0.]

c = [1, 1, 0, 0, 0.]

tableau = build_tableau(A, b, c)

basic_var_list = list(map(lambda x: list(x), list(combinations(list(range(5)), 3))))
for basic_var in basic_var_list:
     print("------- New simplex proccess -------")
     tableau = np.copy(tableau)
     print(f"x_B = {real_index(basic_var)}\n{tableau}")
     correct_all_tableau(tableau, basic_var)
     adjacent_graph(tableau, basic_var, [])

------- New simplex proccess -------
x_B = [1, 2, 3]
[[ 1.  1.  0.  0.  0.  0.]
 [ 1.  2.  1.  0.  0.  5.]
 [ 1. -1.  0.  1.  0.  2.]
 [-1.  3.  0.  0.  1.  0.]]
All tableau corrected
vertex = [3. 1.]	x_B = [1, 2, 3]
[[ 0.   0.   0.  -2.  -1.  -4. ]
 [ 1.   0.   0.   1.5  0.5  3. ]
 [ 0.   1.   0.   0.5  0.5  1. ]
 [-0.  -0.   1.  -2.5 -1.5 -0. ]]
end proccess
------- New simplex proccess -------
x_B = [1, 2, 4]
[[ 0.   0.   0.  -2.  -1.  -4. ]
 [ 1.   0.   0.   1.5  0.5  3. ]
 [ 0.   1.   0.   0.5  0.5  1. ]
 [-0.  -0.   1.  -2.5 -1.5 -0. ]]
All tableau corrected
vertex = [3. 1.], x_B = [1, 2, 4]
[[ 0.   0.  -0.8  0.   0.2 -4. ]
 [ 1.   0.   0.6  0.  -0.4  3. ]
 [ 0.   1.   0.2  0.   0.2  1. ]
 [ 0.   0.  -0.4  1.   0.6  0. ]]
vertex = [3. 1.]	x_B = [1, 2, 5]
[[ 0.          0.         -0.66666667 -0.33333333  0.         -4.        ]
 [ 1.          0.          0.33333333  0.66666667  0.          3.        ]
 [ 0.          1.          0.33333333 -0.33333333  0.          1.        ]
 [ 0